# Challenge 03: Data Modelling: From Retrieval to Upload

In this step, we will structure the data retrieved from Azure Document Intelligence (ADI) into the right format to be read by our systems in subsequent steps. 

The data will be outputted from the ADI as a JSON file, and it is our role to process and organize it. Some of the data will be structured into tables, while other data will be formatted as text. This step ensures that the extracted information is organized in a meaningful way for further analysis and usage.

As stated before, we need to make sure that our Function will know how to process:
- **Loan Forms:** Extract relevant details such as borrower information, loan amounts, and terms.
- **Loan Contract:** Identify and parse key contract elements like clauses, signatures, and dates.
- **Pay Stubs:** Retrieve data such as employee details, earnings, deductions, and net pay.

Not all customers will have provided all types of content, and during this Challenge we will be only be processing one file. We will combine in the next challenge the capabilities of a trigger, which will, at a time, also process one single document.

Due to the nature of this challenge, we will separate this challenge in the 3 different types of documents.

### Terms and Conditions processing

In [12]:
import os
import json
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import re
from azure.cosmos import CosmosClient, exceptions, PartitionKey
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
from openai import AzureOpenAI
# Load environment variables from .env file
load_dotenv()

def read_json_files_from_blob(folder_path):
    # Retrieve the connection string from the environment variables
    connection_string = os.getenv('STORAGE_CONNECTION_STRING')

    # Ensure the connection string is not None
    if connection_string is None:
        raise ValueError("The connection string environment variable is not set.")

    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get the container client
    container_client = blob_service_client.get_container_client("data")

    # List all blobs in the specified folder
    blob_list = container_client.list_blobs(name_starts_with=folder_path)

    # Filter out JSON files and read their contents
    for blob in blob_list:
        if blob.name.endswith('.json'):
            blob_client = container_client.get_blob_client(blob.name)
            blob_data = blob_client.download_blob().readall()
            data = json.loads(blob_data)
            return data 

In [14]:
terms_and_conditions = read_json_files_from_blob("termsandconditions") 

In [25]:
import re

def clean_json_data(json_data):
    """
    Extract relevant text content from the JSON, join it into a plain text string,
    and extract specific fields like Customer Service, Email, and Address.
    """
    content = []

    # Extract text from pages and lines
    pages = json_data.get("pages", [])
    for page in pages:
        for line in page.get("lines", []):
            content.append(line.get("text", "").strip())

    # Join all text content into a single string with spaces between components
    plain_text_content = " ".join(content)

    # Extract Customer Service, Email, and Address using regex
    contact_info = {
        "Customer Service": re.search(r"Customer Service:\s+([\+\d\(\)\-\s]+)", plain_text_content),
        "Email": re.search(r"Email:\s+([\w\.\@]+)", plain_text_content),
        "Address": re.search(r"Address:\s+(.+)", plain_text_content),
    }

    # Clean up the extracted values
    contact_info = {key: (match.group(1).strip() if match else None) for key, match in contact_info.items()}

    return plain_text_content, contact_info


In [28]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-08-01-preview"
)

class TermsAndConditions(BaseModel):
    bank: str
    introduction: str
    loan_amount_and_purpose: str
    interest_rates: str
    loan_tenure: str
    monthly_repayments: str
    late_payments: str
    loan_security: str
    loan_processing_fees: str
    default_and_foreclosure: str
    early_repayment_and_penalties: str
    changes_to_terms: str
    insurance_requirements: str
    loan_cancellation: str
    dispute_resolution: str
    governing_law: str    
    


In [29]:
terms_and_conditions_structured, contact_info = clean_json_data(terms_and_conditions)
completion = client.beta.chat.completions.parse(
    model="gpt-4o", # replace with the model deployment name of your gpt-4o 2024-08-06 deployment
    messages=[
        {"role": "system", "content": "Extract the information about this loan agreement contract."},
        {"role": "user", "content": terms_and_conditions_structured},
    ],
    response_format=TermsAndConditions,
)
final_json_str = completion.model_dump_json(indent=2)

In [30]:
def formatted_data_cleaning(json_string, contact_info):
    """
    Replaces the first 'id' in the JSON string with the 'customer_id' and returns the JSON with only the parsed information.

    Args:
        json_string (str): The original JSON string.

    Returns:
        dict: The modified JSON object with 'id' replaced by 'customer_id' and only the parsed information included.
    """
    # Load the JSON string into a Python dictionary
    data = json.loads(json_string)

    # Extract the parsed information
    parsed_info = data["choices"][0]["message"]["parsed"]

    # Replace the first id with the customer_id
    data["id"] = parsed_info["bank"]

    # Create a new dictionary with only the parsed information
    result = {
        "id": data["id"],
        **contact_info,
        **parsed_info
    }

    return result

# Example usage
result_json = formatted_data_cleaning(final_json_str, contact_info)
print(result_json)

{'id': 'Contoso Bank', 'Customer Service': '+1 (800) 123-4567', 'Email': 'home.loans@contosobank.com', 'Address': 'Contoso Bank, 123 Main Street, City, Country', 'bank': 'Contoso Bank', 'introduction': 'These terms and conditions govern the house loans provided by Contoso Bank (referred to as "the Bank") to customers (referred to as "Borrower"). By applying for and accepting a house loan, the Borrower agrees to the terms and conditions outlined herein.', 'loan_amount_and_purpose': "The loan is granted exclusively for the purpose of purchasing a residential property, refinancing an existing mortgage, or for approved home improvement projects. The maximum loan amount will be determined by the Bank based on the Borrower's financial profile, creditworthiness, and property value.", 'interest_rates': "Fixed Rate: The interest rate remains constant throughout the loan term. Variable Rate: The interest rate may fluctuate based on market conditions and will be tied to a publicly available index

### Code

In [31]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Cosmos DB connection details from environment variables
endpoint = os.getenv("COSMOS_ENDPOINT")
key = os.getenv("COSMOS_KEY")

def upload_text_to_cosmos_db(text_content, container_name):
    # Check if the text is empty
    if not text_content:
        print("The text content is empty. No data to upload.")
        return
    
    # Initialize the Cosmos client
    client = CosmosClient(endpoint, key)
    
    try:
        # Create or get the database
        database = client.create_database_if_not_exists(id="ContosoDB")
        
        # Create or get the container
        container = database.create_container_if_not_exists(
            id=container_name,
            partition_key=PartitionKey(path=f"/id"),
            offer_throughput=400
        )
    except exceptions.CosmosHttpResponseError as e:
        print(f"An error occurred while creating the database or container: {e.message}")
        return
    
    # Create a document with the text content and partition key
    document = {
        'id': text_content.get('id'),  # Generate a unique ID for the document
        'content': text_content,  # Store the plain text as 'content'
    }
    
    # Upload the document to the container
    try:
        container.create_item(body=document)
        print(f"Text content uploaded successfully with ID '{document['id']}' in Cosmos DB.")
    except exceptions.CosmosHttpResponseError as e:
        print(f"An error occurred while uploading the document: {e.message}")

### Upload Pay Stubs

In [32]:
upload_text_to_cosmos_db(result_json, "TermsAndConditions")

Text content uploaded successfully with ID 'Contoso Bank' in Cosmos DB.
